In [1]:
import os
import sys
import time
import serial
import math
import numpy as np

# Adding the src folder in the current directory as it contains the script
# with the Thymio class
sys.path.insert(0, os.path.join(os.getcwd(), 'joachim'))
from Thymio import Thymio
#ls /dev/ttyACM*
#sudo chmod 666 /dev/ttyACM0

In [27]:
class robot():
    def __init__(self):
        self.pos_th = [0,0,0]
        self.th = Thymio.serial(port="/dev/ttyACM0", refreshing_rate=0.1) #/dev/ttyACM0 for linux
        
    def GetPosition(self):
        return self.pos_th

    def measurement_model(self):
        #distance
        x_d = np.linspace(0.5, 5, 10)
        y_d = [16,32.5,46.5,61,79.5,95,109,127.5,143,159.5]
        
        diff_y_d = np.diff(y_d) # Calculating difference list 
        mean_d = np.mean(diff_y_d) #15.94444
        sigma_d = np.std(diff_y_d) #6.877318
        distance_p0 = [1, mean_d, sigma_d]

        #angle
        x_a = np.linspace(0, 9, 10)
        y_a = [0,42,78,114,150,194,231,270,310,347]

        diff_y_a = np.diff(y_a) # Calculating difference list 
        mean_a = np.mean(diff_y_a) 
        sigma_a = np.std(diff_y_a) 
        angle_p0 = [1, mean_a, sigma_a]
        return distance_p0, angle_p0

    def Angle(self,relative_angle):

        if 0 < relative_angle <= math.pi/2: #quadrant 1
            time_tg = (math.degrees(relative_angle)+0.3091)/38.0509
            elf.th.set_var("motor.left.target", 2**16-100) #turn to the left
            self.th.set_var("motor.right.target", 100)
        if math.pi/2 < relative_angle <= math.pi: #quadrant 2
            time_tg = (math.degrees(math.pi-relative_angle)+0.3091)/38.0509
            self.th.set_var("motor.left.target", 2**16-100) #turn to the left
            self.th.set_var("motor.right.target", 100)
        if math.pi < relative_angle <= 1.5*math.pi: #quadrant 3
            time_tg = (math.degrees(relative_angle-math.pi/2)+0.3091)/38.0509
            self.th.set_var("motor.left.target", 100) #turn to the right
            self.th.set_var("motor.right.target", 2**16-100)
        if 1.5*math.pi < relative_angle < 2*math.pi: #quadrant 4     
            time_tg = (math.degrees(relative_angle)+0.3091)/38.0509
            self.th.set_var("motor.left.target", 100) #turn to the right
            self.th.set_var("motor.right.target", 2**16-100) 

        t=0
        t_start = time.time()
        while t < time_tg: #one revolution: 4.378
            t_end= time.time()
            t =  t_end - t_start
        self.th.set_var("motor.left.target", 0)
        self.th.set_var("motor.right.target", 0)

    def Distance(self, d): 
        time_tg = (d+0.4773)/31.809 #time given by the Kalman filter # #d in mm
        self.th.set_var("motor.left.target", 100)
        self.th.set_var("motor.right.target", 100)
        t_start = time.time()
        t=0
        while t < time_tg: #one revolution: 4.378
            t_end= time.time()
            t =  t_end - t_start

        print("time:{}".format(time_tg))
        self.th.set_var("motor.left.target", 0)
        self.th.set_var("motor.right.target", 0)
        time.sleep(0.1)

    def SetPosition(self, p_tg):
        if p_tg == self.pos_th[0:2]: return False #if the robot is already at the position
        #distance
        d = math.sqrt(((self.pos_th[0]-p_tg[0])**2)+((self.pos_th[1]-p_tg[1])**2))
        
        #angle
        if self.pos_th[0]-p_tg[0]==0:
            relative_angle = 0
        else:
            relative_angle = math.atan((p_tg[1]-self.pos_th[1])/(self.pos_th[0]-p_tg[0])) #angle to go to x,y
        new_theta = self.pos_th[2] + relative_angle 
        print("before modulo:{}".format(new_theta))
        new_theta = new_theta%2*math.pi
        print("after modulo:{}".format(new_theta))
        
        print("distance:{}".format(d))
        print("angle:{}".format(new_theta))
        self.Angle(relative_angle)
        self.Distance(d)

        #update position and angle of the robot
        self.pos_th = [p_tg[0],p_tg[1],new_theta]

        return True

In [28]:
th_m = robot()

In [29]:
pos_th = th_m.GetPosition()

distance_p0, angle_p0 = th_m.measurement_model()
print("distance measurement model: {}\n".format(distance_p0))
print("angle measurement model: {}".format(angle_p0))

dir(th_m.th)
th_m.SetPosition([0,50])
pos_th = th_m.GetPosition()
print("Thymio position: {}".format(pos_th))

distance measurement model: [1, 15.944444444444445, 1.6235154662447173]

angle measurement model: [1, 38.55555555555556, 2.753224820747526]
before modulo:0
after modulo:0.0
distance:50.0
angle:0.0


UnboundLocalError: local variable 'time_tg' referenced before assignment

In [41]:
def Main():
    
    a=1
if __name__=="__main__": 
    Main() 